In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
import pathlib

# Defining the image and attributes paths
dataset_path = "img_align_celeba"  
attr_path = "list_attr_celeba.csv" 

# Loading the attributes
df = pd.read_csv(attr_path)
df.set_index("image_id", inplace=True)

# Selecting the desired attributes For example: Smiling
selected_attr = ["Smiling"]
labels = df[selected_attr].replace(-1, 0).astype(np.float32) 

# Loading the dataset
def load_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (64, 64))  # Resizing to the required dimensions
    img = img / 255.0  # Normalize to [0, 1]
    return img, label

# Creating the list of image paths
image_paths = [os.path.join(dataset_path, img) for img in labels.index]
labels = labels.values  # Converting to the numpy array

# Converting to the TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
dataset = dataset.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(64).prefetch(tf.data.AUTOTUNE)

# Test dataset
for images, labels in dataset.take(1):
    print("Batch shape:", images.shape, labels.shape)

# Defining the Generator
def build_generator(latent_dim, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(8 * 8 * 256, activation="relu", input_dim=latent_dim + num_classes),
        tf.keras.layers.Reshape((8, 8, 256)),
        tf.keras.layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", activation="relu"),
        tf.keras.layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", activation="relu"),
        tf.keras.layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding="same", activation="sigmoid")
    ])
    return model

# Defining the Discriminator
def build_discriminator():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, kernel_size=4, strides=2, padding="same", activation="relu", input_shape=(64, 64, 3)),
        tf.keras.layers.Conv2D(128, kernel_size=4, strides=2, padding="same", activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    return model

# Initializing the models
latent_dim = 100
num_classes = 1  # Binary classification that is: Smiling or Not
generator = build_generator(latent_dim, num_classes)
discriminator = build_discriminator()

print("Models created successfully!")


2025-02-07 22:54:30.527555: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Batch shape: (64, 64, 64, 3) (64, 1)
Models created successfully!


2025-02-07 22:54:35.310447: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Defining constants
BATCH_SIZE = 32
NOISE_DIM = 100
NUM_CLASSES = 1 
TOTAL_INPUT_DIM = NOISE_DIM + NUM_CLASSES  

# Generator Model
def build_generator():
    model = keras.Sequential([
        layers.Dense(256, activation="relu", input_shape=(TOTAL_INPUT_DIM,)),
        layers.Dense(512, activation="relu"),
        layers.Dense(1024, activation="relu"),
        layers.Dense(32 * 32 * 3, activation="tanh"),
        layers.Reshape((32, 32, 3))
    ])
    return model

# Discriminator Model
def build_discriminator():
    model = keras.Sequential([
        layers.Flatten(input_shape=(32, 32, 3)),
        layers.Dense(512, activation="relu"),
        layers.Dense(256, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])
    return model

# Creating the models
generator = build_generator()
discriminator = build_discriminator()

# Loss function and the Optimizers
cross_entropy = keras.losses.BinaryCrossentropy()

generator_optimizer = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)

# Training step function
@tf.function
def train_step(real_images, labels):
    batch_size = tf.shape(real_images)[0]

    # Generating the random noise
    noise = tf.random.normal([batch_size, NOISE_DIM])

    # Converting the labels to one-hot encoding and reshape
    labels = tf.one_hot(labels, depth=NUM_CLASSES)
    labels = tf.reshape(labels, (batch_size, NUM_CLASSES))

    # Concatenating the noise and labels correctly
    noise_and_labels = tf.concat([noise, labels], axis=1)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        fake_images = generator(noise_and_labels, training=True)

        real_output = discriminator(real_images, training=True)
        fake_output = discriminator(fake_images, training=True)

        # Calculating the loss
        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
        disc_loss = (cross_entropy(tf.ones_like(real_output), real_output) +
                     cross_entropy(tf.zeros_like(fake_output), fake_output)) / 2

    # Computing the gradients
    gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    # Applying the gradients
    generator_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Training function
def train(dataset, epochs=10):
    for epoch in range(epochs):
        for images, labels in dataset:
            gen_loss, disc_loss = train_step(images, labels)
        print(f"Epoch {epoch+1}, Gen Loss: {gen_loss:.4f}, Disc Loss: {disc_loss:.4f}")

# I am taking an example for a dummy dataset
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()
x_train = (x_train.astype("float32") - 127.5) / 127.5  # Normalizing to [-1, 1]
y_train = y_train.flatten() 
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE)

# Run the training
train(dataset)


2025-02-07 23:04:34.407321: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, Gen Loss: 1.0727, Disc Loss: 0.2762


2025-02-07 23:06:19.460043: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2, Gen Loss: 1.1434, Disc Loss: 0.2611


2025-02-07 23:08:05.685653: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


...

2025-02-07 23:16:54.466823: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8, Gen Loss: 1.8483, Disc Loss: 0.2227


2025-02-07 23:18:38.973022: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 9, Gen Loss: 3.0351, Disc Loss: 0.2042
Epoch 10, Gen Loss: 2.1938, Disc Loss: 0.2058


2025-02-07 23:20:23.316812: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


The GAN training ran smoothly for over 10 epochs. The generator loss started at about 1.07, while the discriminator loss was low at 0.27, which implies that the discriminator was effectively discriminating between real and fake images. In further training, generator loss showed some fluctuations: it reached highs of 7.50 and 8.83 in certain epochs, reflecting moments when the generator struggled to generate realistic samples. It continued relatively stable, but with some variability that could indicate a dynamic competition between the two networks. It is also worth noting that by epoch 7, the discriminator loss had fallen significantly to 0.0387, which may indicate overpowering against the generator. However, in the further epochs, the generator losses have stabilized, and the discriminator loss remained at a moderate value, allowing it to learn the model effectively. It would also be achieved in the results by fine-tuning these hyperparameters, dataset size, or architectural changes that achieve balance and realism.